In [1]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [2]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [3]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@52bbcc44

In [4]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Misc/"
val prefix = "nohup"

val nohup = spark.read.textFile(s"${folder}${prefix}.out")


org.apache.spark.sql.SparkSession$implicits$@75f1d812

In [5]:
println(nohup.count)

360


null

In [6]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, method: String, epsilon: Double, mu: Int, delta: Int)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class DatasetOps


In [8]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.show(10)

+--------------------+---+
|                line|  n|
+--------------------+---+
|FLOCKFINDER=bfe;T...|  0|
|                    |  1|
|      real	2m42.358s|  2|
|      user	2m41.969s|  3|
|        sys	0m0.334s|  4|
|Closing stream: /...|  5|
|totalTime(ms):	16...|  6|
|FLOCKFINDER=bfe;T...|  7|
|FLOCKFINDER=bfe;T...|  8|
|                    |  9|
+--------------------+---+
only showing top 10 rows



null

In [9]:
val indicesRun = lines.filter{ l => 
        l.line.contains("EVENT=Start") || l.line.contains("EVENT=End")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.show(20, truncate = false)

+-----+---+
|runID|n  |
+-----+---+
|0    |0  |
|0    |1  |
|0    |2  |
|0    |3  |
|0    |4  |
|0    |5  |
|0    |6  |
|0    |7  |
|1    |8  |
|1    |9  |
|1    |10 |
|1    |11 |
|1    |12 |
|1    |13 |
|1    |14 |
|1    |15 |
|2    |16 |
|2    |17 |
|2    |18 |
|2    |19 |
+-----+---+
only showing top 20 rows



null

In [10]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr  = line.split(";")
        val method  = arr(0).split("=")(1)
        val epsilon = arr(3).split("=")(1).toDouble
        val mu      = arr(4).split("=")(1).toInt
        val delta   = arr(5).split("=")(1).toInt
        Run(runID, method, epsilon, mu, delta)
    }.
    cache
val nRuns = runs.count()
runs.show(nRuns.toInt, truncate = false)

+-----+------+-------+---+-----+
|runID|method|epsilon|mu |delta|
+-----+------+-------+---+-----+
|0    |bfe   |80.0   |5  |6    |
|1    |bfe   |90.0   |5  |6    |
|2    |bfe   |100.0  |5  |6    |
|3    |bfe   |110.0  |5  |6    |
|4    |bfe   |120.0  |5  |6    |
|5    |bfe   |100.0  |4  |6    |
|6    |bfe   |100.0  |5  |6    |
|7    |bfe   |100.0  |6  |6    |
|8    |bfe   |100.0  |7  |6    |
|9    |bfe   |100.0  |8  |6    |
|10   |bfe   |100.0  |5  |3    |
|11   |bfe   |100.0  |5  |4    |
|12   |bfe   |100.0  |5  |5    |
|13   |bfe   |100.0  |5  |6    |
|14   |bfe   |100.0  |5  |7    |
|15   |bfe   |80.0   |5  |6    |
|16   |bfe   |90.0   |5  |6    |
|17   |bfe   |100.0  |5  |6    |
|18   |bfe   |110.0  |5  |6    |
|19   |bfe   |120.0  |5  |6    |
|20   |bfe   |100.0  |4  |6    |
|21   |bfe   |100.0  |5  |6    |
|22   |bfe   |100.0  |6  |6    |
|23   |bfe   |100.0  |7  |6    |
|24   |bfe   |100.0  |8  |6    |
|25   |bfe   |100.0  |5  |3    |
|26   |bfe   |100.0  |5  |4    |
|27   |bfe

null

In [11]:
val times = lines.join(indexRun, "n").
    filter($"line".rlike("totalTime")).
    map{ m =>
        (m.getInt(2), m.getString(1).split("\t")(1))
    }.
    toDF("runID", "time")
times.show(20, truncate = false)

+-----+-----------+
|runID|time       |
+-----+-----------+
|0    |162080     |
|1    |320030     |
|2    |696550     |
|3    |1.15391e+06|
|4    |1.55363e+06|
|5    |1.19188e+06|
|6    |699980     |
|7    |474060     |
|8    |342150     |
|9    |270160     |
|10   |511290     |
|11   |558310     |
|12   |630360     |
|13   |721660     |
|14   |792870     |
|15   |161510     |
|16   |332700     |
|17   |697720     |
|18   |1.19017e+06|
|19   |1.55315e+06|
+-----+-----------+
only showing top 20 rows



null

In [13]:
val data = runs.join(times, "runID")
val nData = data.count
data.show(nData.toInt, false)

+-----+------+-------+---+-----+-----------+
|runID|method|epsilon|mu |delta|time       |
+-----+------+-------+---+-----+-----------+
|0    |bfe   |80.0   |5  |6    |162080     |
|1    |bfe   |90.0   |5  |6    |320030     |
|2    |bfe   |100.0  |5  |6    |696550     |
|3    |bfe   |110.0  |5  |6    |1.15391e+06|
|4    |bfe   |120.0  |5  |6    |1.55363e+06|
|5    |bfe   |100.0  |4  |6    |1.19188e+06|
|6    |bfe   |100.0  |5  |6    |699980     |
|7    |bfe   |100.0  |6  |6    |474060     |
|8    |bfe   |100.0  |7  |6    |342150     |
|9    |bfe   |100.0  |8  |6    |270160     |
|10   |bfe   |100.0  |5  |3    |511290     |
|11   |bfe   |100.0  |5  |4    |558310     |
|12   |bfe   |100.0  |5  |5    |630360     |
|13   |bfe   |100.0  |5  |6    |721660     |
|14   |bfe   |100.0  |5  |7    |792870     |
|15   |bfe   |80.0   |5  |6    |161510     |
|16   |bfe   |90.0   |5  |6    |332700     |
|17   |bfe   |100.0  |5  |6    |697720     |
|18   |bfe   |110.0  |5  |6    |1.19017e+06|
|19   |bfe

null

In [14]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}bfe_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null